In [36]:
#!/usr/bin/python

In [37]:
# Importamos las librerias necesarias desde bash para el desarrollo de este codigo

!git clone https://github.com/st1800eafit/st1800_20211.git
!pip install nltk
!pip install stop-words
!pip install pandas
!pip install gensim

fatal: destination path 'st1800_20211' already exists and is not an empty directory.


In [38]:
# Importamos las librerias necesarias desde Python para el desarrollo de este codigo

import os
import glob
import re 
import nltk 
import pandas as pd
import numpy as np

In [39]:
# Dede la libreria de NLTK necesitaremos estos parametros para realizar la limpieza o preparacion de los documentos

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
# Abrimos los documentos .txt que en este caso utilizaremos y adicionalmente, abrimos el documento de xml. A este ultimo se le realizara una limpieza antes de tokenizar. Para asi remover las etiquetas

files_location = os.path.join("/","content","st1800_20211","datasets", "papers_sample_pdf/")
output_file_location = os.path.join("/","content/")
output_file = "scikit_model_vectorized.sav"

filenames = glob.glob(files_location+"*.txt")
corpus_per_document = []

for file in filenames:
  doc_corpus = open(file, "r").read()
  corpus_per_document.append(doc_corpus)

# Adicionamos el XML sin tags
filename_xml = glob.glob(files_location+"*.xml")
xml_file = open (filename_xml[0], "r").read()
xml_file = re.sub('<[^>]*>', "", xml_file)
corpus_per_document.append(xml_file)

In [41]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim import corpora, models

# Aplicaremos la tecnica de TF-IDF pero en Gensim. Primero tokenizaremos los documentos con NLTK

Porter_Stemmer = PorterStemmer()

cleaned_corpus_per_document = []
stop_words_nltk = set(stopwords.words('english'))

#Primero preparamos cada elemento del corpus
for document in corpus_per_document:
  clean_document = nltk.word_tokenize(document)
  clean_document = [re.sub(r'[^A-Za-z0-9]+','',token) for token in clean_document]
  clean_document = [token.lower() for token in clean_document if len(token)>1]
  clean_document = [token for token in clean_document if token not in stop_words_nltk]
  clean_document = [Porter_Stemmer.stem(token) for token in clean_document]
  cleaned_corpus_per_document.append(clean_document)

#Luego aplicamos Gensim creando el bag of words
bow = corpora.Dictionary(cleaned_corpus_per_document)
#print(bow.token2id)
corpus = [bow.doc2bow(text) for text in cleaned_corpus_per_document]

#Con este mecanismo podremos guardar el corpus procesado
corpora.MmCorpus.serialize('/content/docs_corpus.mm', corpus)

#Ahora aplicaremos TF-IDF. Podemros obesrvar que el resultado seria una pareja, con el numero de ocurrencias y el valor del IDF
tfidf = models.TfidfModel(corpus)

#for document in tfidf[corpus]:
#  print(document)

for i in range (6):
  print(tfidf[corpus][i][0:5])


[(0, 0.010450743622335182), (1, 0.0019285292620283823), (2, 0.00336129584950534), (3, 0.005225371811167591), (4, 0.0071720187885617225)]
[(6, 0.00684812475308699), (48, 0.006262237830021386), (49, 0.0009783035361552843), (59, 0.002087412610007129), (84, 0.0009783035361552843)]
[(1, 0.0022906146563204596), (2, 0.009648266556126443), (3, 0.0031032231377177088), (4, 0.0369138479059912), (5, 0.015516115688588544)]
[(1, 0.0008246360431166116), (2, 0.0019163796741599247), (4, 0.002044495879634551), (6, 0.003593211889049859), (11, 0.002044495879634551)]
[(2, 0.0007132080749496942), (20, 0.0015217767024125251), (46, 0.0015217767024125251), (47, 0.00736560203941613), (54, 0.0021396242248490826)]
[(1, 0.0049914801450228405), (2, 0.0018124607769656727), (4, 0.002320355599970522), (6, 0.0025374450877519417), (11, 0.002320355599970522)]


In [42]:
# Primeros ubicamos nuestro corpus en un dataframe para nuestra siguiente implementacion

df = pd.DataFrame(cleaned_corpus_per_document)
df_clean = df.dropna().drop_duplicates()
final_bow = df_clean.values.tolist()


In [43]:
# Utilizaremos la libreria de word2Vec de Gensim y adicionalmente, evidenciaremos cuantos nucleos tenemos disponibles en nuestro procesamiento
import multiprocessing

from gensim.models import Word2Vec

cores = multiprocessing.cpu_count()

cores

2

In [44]:
# Creamos el modelo con los siguientes parametros, entre ellos que solo considere las palabras con minimmo 5 ocurrencias, un ritmo de aprendizaje de 0.03, entre otrods.
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [45]:
# Permitimos que el modelo adquiera el vocabulario ingresado
w2v_model.build_vocab(final_bow, progress_per=10000)


In [46]:
# Entrenamos el modelo y evidenciamos el tiempo que se demoro en entrenarse.
w2v_model.train(final_bow, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(300000, 3846450)

In [47]:
# Finalmente podremos guardar los parametros del modelo
w2v_model.init_sims(replace=True)

In [48]:
# Probamos el modelo. Aqui podremos observar que dada la plabra comput, la palabra que mas probabilidad tiene para que le siga es science, lo cual tiene sentido.
w2v_model.wv.most_similar(positive=["comput"])

[('scienc', 0.9999433159828186),
 ('mathemat', 0.9999307990074158),
 ('combinator', 0.9999194741249084),
 ('discret', 0.9999142289161682),
 ('theori', 0.9999040961265564),
 ('geometri', 0.9999023079872131),
 ('structur', 0.9998992681503296),
 ('parallel', 0.999896228313446),
 ('michael', 0.999894380569458),
 ('68q17', 0.9998940825462341)]

In [49]:
# Asi podremos guardar el modelo para no tener que realizarlo nuevamente
w2v_model.save("/content/gensim_model.model")

# Adicionalmente de esta manera podremos cargarlo para nuestro analisis
loaded_w2d_model = Word2Vec.load("/content/gensim_model.model")